In [1]:
import numpy as np
import pandas as pd

def vasicek_irb(r, a, b, sigma, t, n=12):
    """
    Function to simulate default probabilities using the Vasicek IRB model
    """
    dt = t/n
    r_sim  = np.zeros(n+1)
    r_sim [0] = r
    for i in range(1, n+1):
        r_sim [i] = r_sim[i-1] + a*(b-r_sim[i-1])*dt + sigma*np.sqrt(dt)*np.random.normal()
    return r_sim

# Read loan data from CSV file
loan_data = pd.read_csv("loan_data.csv")

# Assign parameter values for the Vasicek IRB model
a = 0.1
b = 0.04
sigma = 0.02

# Calculate default probabilities for each loan
default_probs = []
for index, row in loan_data.iterrows():
    r = row["Interest Rate"]
    t = row["Loan Term"]
    r_sim = vasicek_irb(r, a, b, sigma, t)
    default_prob = np.mean(r_sim[-1] < r)
    default_probs.append(default_prob)

# Add default probabilities to loan data
loan_data["Default Probability"] = default_probs

# Print results
print(loan_data)


   Interest Rate  Loan Term  Default Rate  Default Probability
0           0.05          5          0.03                  0.0
1           0.06         10          0.05                  1.0
2           0.07         15          0.08                  0.0
3           0.08         20          0.10                  0.0
4           0.09         25          0.11                  1.0
5           0.10         30          0.12                  0.0
6           0.11         35          0.16                  1.0


In [2]:
from scipy.optimize import minimize

# Define a function to calculate the error between simulated and actual default rates
def calc_error(params, loan_data):
    a, b, sigma = params
    default_probs = []
    for index, row in loan_data.iterrows():
        r = row["Interest Rate"]
        t = row["Loan Term"]
        r_sim = vasicek_irb(r, a, b, sigma, t)
        default_prob = np.mean(r_sim[-1] < r)
        default_probs.append(default_prob)
    error = np.mean((default_probs - loan_data["Default Rate"])**2)
    return error

# Initial guess for parameters
params_init = [0.1, 0.04, 0.02]

# Optimize parameters
params_opt = minimize(calc_error, params_init, args=(loan_data,), method='BFGS')

# Extract optimized parameter values
a_opt, b_opt, sigma_opt = params_opt.x

# Use the optimized parameters to calculate default probabilities
default_probs = []
for index, row in loan_data.iterrows():
    r = row["Interest Rate"]
    t = row["Loan Term"]
    r_sim = vasicek_irb(r, a_opt, b_opt, sigma_opt, t)
    default_prob = np.mean(r_sim[-1] < r)
    default_probs.append(default_prob)

# Add default probabilities to loan data
loan_data["Default Probability"] = default_probs

# Print results
print(loan_data)


   Interest Rate  Loan Term  Default Rate  Default Probability
0           0.05          5          0.03                  1.0
1           0.06         10          0.05                  1.0
2           0.07         15          0.08                  0.0
3           0.08         20          0.10                  0.0
4           0.09         25          0.11                  1.0
5           0.10         30          0.12                  1.0
6           0.11         35          0.16                  1.0
